<a href="https://colab.research.google.com/github/pijuma/Facul/blob/main/algelin/Filtragem_em_Sistemas_de_Recomenda%C3%A7%C3%A3o_com_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Filtragem em Sistemas de Recomendação com Decomposição em Valores Singulares (SVD)

# Alunos

Pietra Gullo Salgado Chaves - 14603822

Vinicius Gustierrez Neves - 14749363

Clara Ernesto de Carvalho - 14559479

Neste trabalho, utilizamos um subconjunto do One Million Song Dataset, um grande banco de dados contendo informações detalhadas sobre as músicas e as preferências de mais de um milhão de usuários. Este dataset foi criado para facilitar pesquisas em áreas como recomendação de músicas, análise de preferências e estudo de comportamento do usuário. A versão completa do dataset pode ser acessada no link [http://millionsongdataset.com/tasteprofile/#getting].

Este dataset contém conjuntos de códigos que representam os diferentes usuários e códigos que representam as diferentes músicas ouvidas pelos usuários. Dessa forma, os dados permitem a obtenção da relação entre os agentes citados.

Devido ao tamanho extenso do dataset completo (280-300GB), optamos por utilizar uma amostra representativa, correspondendo a apenas 0,1% dos dados originais. O arquivo anexado ao trabalho contém essa amostra selecionada.

O objetivo principal deste trabalho é identificar as X músicas mais recomendadas para cada usuário, com base nas preferências passadas (quantas vezes ele escutou outras músicas no banco de dados). O sistema de recomendação foi desenvolvido para sugerir músicas que um usuário provavelmente gostaria de ouvir, mesmo que ele ainda não tenha interagido com elas. Para isso, seguimos os seguintes passos:

* **Pré-processamento dos Dados:** Inicialmente, mapeamos os IDs dos usuários e das músicas para índices inteiros, criando uma estrutura de dados que permite a fácil manipulação e análise posterior.

* **Construção da Matriz de Interações:** Com os dados organizados, geramos uma matriz em que cada linha representa um usuário e cada coluna uma música. O valor em cada célula dessa matriz representa o número de vezes que o usuário ouviu a música correspondente. Denominaremos essa matriz de matriz $P$.

* **Decomposição em Valores Singulares (SVD):** A matriz de interações foi então analisada utilizando a técnica de Decomposição em Valores Singulares (SVD). Essa técnica permite reduzir a dimensionalidade dos dados, extraindo padrões latentes entre os usuários e as músicas. A SVD nos fornece uma representação de usuários e músicas em um espaço de menor dimensão, destacando os fatores que influenciam as preferências dos usuários.

* **Cálculo de Similaridades**: Utilizamos os fatores latentes extraídos pela SVD para calcular a similaridade entre os usuários e as músicas. A similaridade foi medida utilizando o cosseno de similaridade, que nos permite identificar quais músicas são mais alinhadas com os gostos de um usuário.

* **Geração das Recomendações:** Finalmente, com base nas similaridades calculadas, o sistema gera as X músicas mais recomendadas para cada usuário, priorizando aquelas que tem maior chance de agradá-lo, dentre as que ele não escutou.

O código e seus passos e contextos serão expostos abaixo:

In [ ]:
#biblioteca utilizadas
import numpy as np
import pandas as pd

1) Inicialmente, precisamos importar nossos dados e mapear os ID's dos usuários e os nome das músicas para valores inteiros, tornando mais fácil a indexação dos dados durante os próximos passos do processo.

In [ ]:
data = pd.read_csv("sampled_triplets.txt", delimiter="\t", skiprows=1, names=["user_id", "song_id", "play_count"])

#mapear os ID's dos usuários e nome das músicas para inteiros
user_mapping = {id: idx for idx, id in enumerate(data['user_id'].unique())}
song_mapping = {id: idx for idx, id in enumerate(data['song_id'].unique())}

#substituir pelos indices inteiros
data['user_idx'] = data['user_id'].map(user_mapping)
data['song_idx'] = data['song_id'].map(song_mapping)

print(data)
#aqui temos os dados de cima como estavam no dataset
#e os dados abaixo depois do mapeamento

                                       user_id             song_id  \
0     7dd192c8bd4f27f573cb15e8656442aadd7a9c01  SOYMLDU12AAF3B22A6   
1     3f9a8180776f260cd5bd933e6c49085c687856a2  SORYLMG12A8C1339F9   
2     68c08b200edaccd6e6758b8a1c9236322a4a776d  SOLZART12AB01858FC   
3     cc737b71cd40703432b789527f7811fdc2beec8b  SOTOFDZ12A6D4FC0C7   
4     0abab34c46d6b9242720bf55e50e325a911d9d3a  SOXOMQQ12A8C13BF49   
...                                        ...                 ...   
4832  d7e61fd11949f27395c0b2ad08d6135587a895bb  SOMIZHX12AC468BF5B   
4833  8ac25e0294046e58b471f3d7fdc749d7e8a8614e  SOULTKQ12AB018A183   
4834  d61d3f131c5831d4a4e552af3cdd23217b752a7c  SOGREGV12A58A77030   
4835  e299a35e4721b2c83ebaa2fee881229657edd43e  SOCGGKK12AB017F094   
4836  274d88a609f780d0abc9d3258fde657ad59d55dd  SOZEGZO12A58A7C25E   

      play_count  user_idx  song_idx  
0              2         0         0  
1              9         1         1  
2              1         2         2  
3  

2) Agora que temos os dados, precisamos construir nossa matriz P, em que cada linha irá simbolizar um usuário e cada coluna uma música. Cada posição P[i][j] irá guardar quantas vezes a música j foi escutada pelo usuário i. Dessa forma, teremos uma relação de valores inteiros entre os dados que anteriormente eram representados por códigios do dataset.

In [ ]:
N = len(user_mapping) #numero de usuários
M = len(song_mapping) #numero de músicas

P = np.zeros((N, M), dtype=int) #criação da matriz de usuários por música

#percorrer cada linha do dataset para criar a matriz para usarmos no SVD
for linha in data.iterrows():
    row = linha[1]
    P[row['user_idx'], row['song_idx']] = row['play_count']

#A matriz que iremos aplicar SVD e as dimensões dela
print(f'Dimensões da matriz P: {N}x{M}\n')
print(P)

Dimensões da matriz P: 4806x4218

[[2 0 0 ... 0 0 0]
 [0 9 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 5]]


In [ ]:
"""É necessário montar a matriz de interação, que será usada nas próximas etapas para evitar a
redundância das recomendações obtidas. Para cada usuário e cada música teremos 1 se ele já escutou e 0 caso contrário"""

intt = [[0 for _ in range(M)] for _ in range(N)]

for i in range(N):
  for j in range(M):
    intt[i][j] = 1 if P[i][j] > 0 else 0

Nosso objetivo agora é encontrar os **fatores latentes**, ou seja, as características que capturam padrões nos nossos dados pelas direções principais de variação. Uma vez que agora temos a matriz de base, iremos aplicar **SVD (Decomposição em Valores Singulares)** nela.

Para isso, iremos encontrar os **autovalores** de $P P^T$, bem como os **autovetores** de $P^T P$ e $P P^T$, para assim descobrirmos a intensidade de cada fator latente para cada usuário, bem como para cada música.

Ao final do cálculo do SVD, obteremos as matrizes $U_k$, $V_k^T$ e $\Sigma$, que representam, respectivamente:

* **$U_k$**: A matriz dos usuários representados em termos de fatores latentes, considerando apenas os **k fatores latentes mais relevantes**. Cada linha da matriz $U_k$ representa um usuário no novo espaço de fatores latentes, e as colunas representam a intensidade de cada fator latente para esse usuário. Assim, $U_k$ nos dá uma representação compacta dos usuários, focando nas características mais importantes que influenciam suas preferências.

* **$V_k^T$**: A matriz das músicas representadas em termos de fatores latentes, considerando apenas os **k fatores latentes mais relevantes**. Cada linha de $V_k^T$ corresponde a uma música, e as colunas indicam a intensidade de cada fator latente para essa música. A matriz $V_k^T$ captura as características latentes das músicas, permitindo que músicas com perfis similares tenham representações vetoriais próximas.

* **$\Sigma$**: A matriz diagonal com os **autovalores**, que indicam a importância de cada fator latente. Os autovalores em $\Sigma$ refletem a quantidade de variabilidade nos dados que é explicada por cada fator latente. Quanto maior o autovalor, mais importante é o fator latente na explicação das preferências dos usuários e músicas. Valores pequenos podem ser descartados, pois indicam que os fatores latentes correspondentes não acrescentam muito ao modelo.

### Matemática por Trás da Decomposição

A decomposição de SVD busca fatorar a matriz de interações $P$, que representa quantas vezes cada usuário escutou a cada música, como o produto das matrizes $U$, $\Sigma$ e $V^T$, ou seja:

$$
P = U \Sigma V^T
$$

- **$U$** contém os autovetores de $P P^T$, e seus elementos representam os usuários no espaço latente.
- **$\Sigma$** é uma matriz diagonal, contendo os autovalores que representam a importância de cada fator latente.
- **$V^T$** contém os autovetores de $P^T P$, representando as músicas no espaço latente.

Ao calcular a SVD e reduzir a dimensionalidade para $k$ fatores latentes mais relevantes, a matriz resultante preserva a maior parte da variabilidade nos dados originais, ao mesmo tempo em que elimina o "ruído" e componentes menos significativos.

Para calcular as matrizes **$U$** e **$V^T$** bem como **$\Sigma$** nós utilizaremos da matriz $P P^T$.

De forma que:
$P P^T$ = $(U \Sigma V^T)*(U \Sigma V^T)^T$ = $U\Sigma$ $V^TV\Sigma U^T$ = $U \Sigma U^T$

Portanto, para obtermos os autovalores que esperamos devemos pegar a raíz quadrada dos autovalores obtidos de $P P^T$. Temos que para $P^TP$ é análogo.

3) Aplicando o SVD na Matriz V

In [ ]:
#Calcular a matriz de covariância e obter os valores singulares e vetores
mult = np.dot(P, P.T)
sigma, U = np.linalg.eig(mult)

# Ordenar os valores singulares e autovetores
sorted_indices = np.argsort(sigma)[::-1]
sigma = sigma[sorted_indices]
sigma[sigma < 0] = 0  # Definir valores negativos como zero (em caso de pequenas imprecisões)
sigma = np.sqrt(sigma)  # Valores singulares são a raiz quadrada dos autovalores
U = U[:, sorted_indices]  # Reordene as colunas de U

sigma[sigma < 1e-10] = 1e-10  # Substituir valores muito pequenos por um valor mínimo
V = np.dot(U.T, P) / sigma[:, np.newaxis]
k = 500 #Quantos fatores de latência queremos levar em consideração

# Reduzir as dimensões de U, sigma e V para k
U_k = U[:, :k]  # m x k
sigma_k = np.diag(sigma[:k])  # k x k
VT_k = V[:k, :] # k x n

Agora que temos todas as matrizes que precisávamos, podemos prosseguir com a recomendação de músicas. O próximo passo é calcular a **similaridade entre os usuários e as músicas**. Para isso, usamos os autovetores das matrizes $U_k$ e $V_k^T$ que obtivemos na decomposição SVD.

Cada **linha de $U_k$** representa a importância de cada fator latente para um usuário, enquanto cada **coluna de $V_k^T$** representa a intensidade de cada fator latente para uma música.

### Cálculo da Similaridade

Para determinar a similaridade entre um usuário e uma música, vamos calcular o **cosseno do ângulo** formado entre o vetor de características latentes do usuário (representado por uma linha de $U_k$) e o vetor de características latentes da música (representado por uma coluna de $V_k^T$). Esse cálculo nos diz o quão próximo está o perfil do usuário em relação ao perfil de cada música.

Quanto maior o valor do cosseno, mais próximos estão os vetores de características, o que significa que o usuário provavelmente gostará da música.

### Fórmula do Cálculo do Cosseno

O cálculo do **cosseno de similaridade** entre o vetor de um usuário $u$ e o vetor de uma música $v$ é dado pela fórmula:

$$
\text{coseno}(u, v) = \frac{u \cdot v}{\|u\| \|v\|}
$$

Onde:
- $u \cdot v$ é o **produto escalar** entre os dois vetores.
- $\|u\|$ e $\|v\|$ são as **normas** (ou magnitudes) dos vetores $u$ e $v$, respectivamente. Sendo essas calculadas por $sqrt(<u, v>)$.

### Interpretação

- **Produto escalar ($u \cdot v$)**: Mede a correlação entre os vetores. Quanto maior esse valor, mais próximos são os perfis de preferências entre o usuário e a música.
- **Normas ($\|u\|$ e $\|v\|$)**: Garantem que a similaridade seja normalizada, ou seja, ela leva em consideração apenas o ângulo entre os vetores, não suas magnitudes. Assim, se o vetor de um usuário for muito maior em magnitude do que o da música, isso não afetará o resultado da similaridade.

### Passos para Recomendação

- Para cada usuário, calculamos o cosseno de similaridade entre seu vetor de características latentes e o vetor de cada música.
- As músicas que apresentarem os maiores valores de cosseno de similaridade são as mais prováveis de agradar ao usuário.
- Para identificar as **X músicas mais recomendadas**, basta ordenar os valores de similaridade de forma decrescente e selecionar os maiores valores.

Com esse processo, conseguimos sugerir as músicas que têm maior chance de serem do gosto do usuário, baseando-se nas características latentes identificadas pela SVD.


4) Iremos definir as X músicas mais prováveis do usuário Y gostar para cada usuário.

In [ ]:
def recomendar_musicas(user_idx, U_k, VT_k, interacoes, song_mapping, X=5):
    user_vector = U_k[user_idx] #obter o vetor latente para o user_idx
    #queremos calcular <usuario,musica>/<usuario,usuario>*<musica,musica> para achar o cossenos e poder pegar os mais proximos
    norma_user = np.linalg.norm(user_vector)
    norma_v = np.linalg.norm(VT_k, axis=0)
    cossenos = np.zeros(VT_k.shape[1])  # Inicializar com zeros
    if norma_user > 0:
        cossenos = np.dot(VT_k.T, user_vector) / (norma_v * norma_user + 1e-10)
    cossenos[interacoes[user_idx]>0] = -2 #todas músicas que o usuario já escutou eu não quero escolher
    ordenado = np.argsort(cossenos)[::-1] #quero ordenar os cossenos de forma decrescente (armazena os ids)
    ct = 0
    melhores_ids = []
    ct = 0
    for i in ordenado:
        if cossenos[i] == -2:  # Ignora entradas inválidas definidas como -2 (músicas já escutadas)
            break
        song_id = list(song_mapping.keys())[list(song_mapping.values()).index(i)]
        melhores_ids.append(song_id)
        ct += 1
        if ct == X:
            break
    return melhores_ids

Agora, temos a **visualização dos dados** abaixo, onde podemos ver, para cada usuário, o código de identificação do usuário seguido das músicas recomendadas para ele. Cada usuário tem um conjunto de músicas que foram sugeridas com base na análise de similaridade entre o perfil do usuário e as características latentes das músicas.

A saída está organizada da seguinte forma:

- **Código do usuário**: A identificação única de cada usuário, representado por um número.
- **Músicas recomendadas**: A lista das **X músicas mais recomendadas** para o usuário, com base no cálculo de similaridade realizado pelos fatores latentes. Essas músicas foram selecionadas por sua proximidade ao perfil do usuário, de acordo com os fatores latentes extraídos da decomposição SVD.



In [ ]:
#para facilitar visualização iremos encontrar para os 4 primeiros usuarios apenas
ct = 0
for i, linha in enumerate(U_k):
    resp = recomendar_musicas(i, U_k, VT_k, intt[i], song_mapping)
    user_id = list(user_mapping.keys())[list(user_mapping.values()).index(i)]
    if len(resp) != 0:
      print(f"Para o usuário {user_id}, temos as recomendações:")
      ct+=1
    for j in resp:
      print(j)
    if ct == 4:
      break

Para o usuário 6ccd111af9b4baa497aacd6d1863cbf5a141acc6, temos as recomendações:
SODCNJX12A6D4F93CB
SOBEVGM12A67ADBCA7
SOSVMCJ12A6310E292
SOHXBXX12B34F2609B
SOLVNYY12AB0181D2E
Para o usuário ba199b26c1ce4a9bab8c7d3cbdd8e6c0dff9fdab, temos as recomendações:
SOZEGZO12A58A7C25E
SOJZRVM12AB0186D67
SOBCZDE12A6D4F87E4
SOKSNTY12A8C1380FB
SOLWHOJ12A67020685
Para o usuário e5f76977ade55eea9b3ea5aba3be0fc984bd1767, temos as recomendações:
SOZEGZO12A58A7C25E
SOJZRVM12AB0186D67
SOBCZDE12A6D4F87E4
SOKSNTY12A8C1380FB
SOLWHOJ12A67020685
Para o usuário 2ddcdf3934856bd7bdc0efd4c234e42b9b5f9e06, temos as recomendações:
SOZEGZO12A58A7C25E
SOJZRVM12AB0186D67
SOBCZDE12A6D4F87E4
SOKSNTY12A8C1380FB
SOLWHOJ12A67020685
